In [1]:
!unzip '/content/drive/MyDrive/데이콘/제주 가격 예측/open.zip'

Archive:  /content/drive/MyDrive/데이콘/제주 가격 예측/open.zip
   creating: open/
  inflating: open/international_trade.csv  
  inflating: open/sample_submission.csv  
  inflating: open/test.csv           
  inflating: open/train.csv          


In [ ]:
# !pip install autogluon

In [3]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from autogluon.tabular import TabularDataset, TabularPredictor

In [4]:
train_df = pd.read_csv('/content/open/train.csv')
test_df = pd.read_csv('/content/open/test.csv')

In [13]:
test_df.head()

,ID,timestamp,item,corporation,location,year,month,day
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8


In [14]:
len(train_df)

59397

In [6]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
train_df['year'] = train_df['timestamp'].apply(lambda x : int(x[0:4]))
train_df['month'] = train_df['timestamp'].apply(lambda x : int(x[5:7]))
train_df['day'] = train_df['timestamp'].apply(lambda x : int(x[8:10]))

test_df['year'] = test_df['timestamp'].apply(lambda x : int(x[0:4]))
test_df['month'] = test_df['timestamp'].apply(lambda x : int(x[5:7]))
test_df['day'] = test_df['timestamp'].apply(lambda x : int(x[8:10]))

In [7]:
#학습에 사용하지 않을 변수들을 제거합니다
train_x = train_df.drop(columns=['ID', 'timestamp', 'supply(kg)'])

test_x = test_df.drop(columns=['ID', 'timestamp'])

In [8]:
qual_col = ['item', 'corporation', 'location']

for i in qual_col:
    le = LabelEncoder()
    train_x[i]=le.fit_transform(train_x[i])
    test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

print('Done.')

Done.


In [9]:
train_data = TabularDataset(train_x)
save_path = 'automodel'
predictor =  TabularPredictor(
    label = 'price(원/kg)',
    path=save_path,
    eval_metric = 'root_mean_squared_error',
    problem_type = 'regression'
)
predictor.fit(
    train_data = train_data,
    presets = 'best_quality',
    time_limit = 60*100
)

	-1249.3765	 = Validation score   (-root_mean_squared_error)
	852.64s	 = Training   runtime
	2.38s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ... Training model for up to 666.79s of the 2667.64s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	-1009.8749	 = Validation score   (-root_mean_squared_error)
	544.77s	 = Training   runtime
	314.89s	 = Validation runtime
Fitting model: NeuralNetTorch_BAG_L1 ... Training model for up to 90.85s of the 2091.7s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	-1500.6682	 = Validation score   (-root_mean_squared_error)
	97.29s	 = Training   runtime
	1.23s	 = Validation runtime
Completed 1/20 k-fold bagging repeats ...
Fitting model: WeightedEnsemble_L2 ... Training model for up to 399.87s of the 1990.1s of remaining time.
	-979.4841	 = Validation score   (-root_mean_squared_error)
	2.96s	 = Training   runtime
	0.0s	 = Validatio

In [10]:
preds = predictor.predict(test_x)

In [11]:
submission = pd.read_csv('/content/open/sample_submission.csv')
submission
submission['answer'] = preds
submission

,ID,answer
0,TG_A_J_20230304,3725.693359
1,TG_A_J_20230305,2316.454834
2,TG_A_J_20230306,944.171448
3,TG_A_J_20230307,3508.639404
4,TG_A_J_20230308,3365.193115
...,...,...
1087,RD_F_J_20230327,353.664368
1088,RD_F_J_20230328,519.358582
1089,RD_F_J_20230329,342.513397
1090,RD_F_J_20230330,557.878357


In [12]:
submission.to_csv('./baseline_submission.csv', index=False)